In [ ]:
def validate_table(table, unique_column, conn_sf, conn_db, db_name):
    cur_sf = conn_sf.cursor()
    cur_db = conn_db.cursor()

    result = {"table": table, "failures": []}

    try:
        # Row counts
        sf_count = fetch_scalar(cur_sf, f"SELECT COUNT(*) FROM {table}")
        db_count = fetch_scalar(cur_db, f"SELECT COUNT(*) FROM {db_name}.{table.lower()}")
        if sf_count != db_count:
            result["failures"].append({"check": "row_count_match", "reason": f"Snowflake: {sf_count}, Databricks: {db_count}"})
    except Exception as e:
        result["failures"].append({"check": "row_count_match", "reason": str(e)})

    try:
        # Unique values
        sf_unique = fetch_scalar(cur_sf, f"SELECT COUNT(DISTINCT {unique_column}) FROM {table}")
        db_unique = fetch_scalar(cur_db, f"SELECT COUNT(DISTINCT {unique_column}) FROM {db_name}.{table.lower()}")
        if sf_unique != db_unique:
            result["failures"].append({"check": "unique_values_match", "reason": f"Snowflake: {sf_unique}, Databricks: {db_unique}"})
    except Exception as e:
        result["failures"].append({"check": "unique_values_match", "reason": str(e)})

    try:
        # Schema
        sf_cols = fetch_column_names(cur_sf, f"DESCRIBE TABLE {table}")
        db_cols = fetch_column_names(cur_db, f"DESCRIBE TABLE {db_name}.{table.lower()}")
        if set(sf_cols) != set(db_cols):
            result["failures"].append({"check": "schema_match", "reason": f"Columns mismatch. Snowflake: {sf_cols}, Databricks: {db_cols}"})
    except Exception as e:
        result["failures"].append({"check": "schema_match", "reason": str(e)})

    cur_sf.close()
    cur_db.close()
    return result
